# Določevanje razpoloženja (sentimenta)

## Namestitev programskih modulov

In [ ]:
!pip install transformers
!pip install python-docx
!pip install ipywidgets

## Priklic programskih modulov

In [1]:
from transformers import pipeline
from docx import Document
import re


## Priklic besedila

In [2]:
# Naloži dokument
file_path = '/content/Antrittsrede_von_Donald_Trump_englisch.docx'
# file_path = r"data\Trump_inauguration\Antrittsrede_von_Donald_Trump_englisch.docx"
doc = Document(file_path)


## Priprava besedila

V prvem koraku pridobimo besedilo iz naložene Wordove datoteke.

V drugem koraku razdelimo besedilo na povedi. Merilo segmentacije so končna ločila. Pri tem opravilu nam pomagajo regularni izrazi.

In [3]:
# Izvleči besedilo iz dokumenta
text = "\n".join([p.text for p in doc.paragraphs if p.text.strip()])
sentences = re.split(r'(?<=[.!?]) +', text)


## Priklic sentimentnega modela

In [4]:
# Naloži model sentimentne analize
sentiment_analysis = pipeline("sentiment-analysis", model="cardiffnlp/twitter-xlm-roberta-base-sentiment")


## Analiza besedila

In [5]:
# Analiziraj sentiment posamezne povedi
results = [{"sentence": sentence, "sentiment": sentiment_analysis(sentence)[0]} for sentence in sentences]


## Prikaz izidov

In [6]:
# Prikaži rezultate
for result in results:
    print(f"Sentence: {result['sentence']}")
    print(f"Sentiment: {result['sentiment']['label']} (Score: {result['sentiment']['score']})\n")


Sentence: It is my honor and pleasure to introduce to you the 45th and the 47th President of the United States of America, Donald J.
Sentiment: positive (Score: 0.8757715225219727)

Sentence: Trump.
Sentiment: negative (Score: 0.4194532632827759)

Sentence: Thank you very much, everybody.
Sentiment: positive (Score: 0.9117685556411743)

Sentence: Well, thank you very, very much.
Sentiment: positive (Score: 0.8509612083435059)

Sentence: Vice President Vance, Speaker Johnson, Senator Thune, Chief Justice Roberts, Justices of the United States Supreme Court, President Clinton, President Bush, President Obama, President Biden, Vice President Harris, and my fellow citizens.
Sentiment: neutral (Score: 0.6428961753845215)

Sentence: Thank you.
Sentiment: positive (Score: 0.8660821914672852)

Sentence: The golden age of America begins right now.
Sentiment: positive (Score: 0.7755902409553528)

Sentence: From this day forward, our country will flourish and be respected again all over the world

## Sentiment v barvah

S funkcijo get_color() obarvamo povedi besedila obarvamo glede na vrednost sentimenta (positive, neutral, negative).

Nato ustvarimo osnovo za spletno stran v ustreznem oblikovnem slogu (naslov, nabor in velikost črk, velikost strani, barve).

Sledi dodeljevanje izbranih barv (green, gray, red) posameznim povedim glede na izračunano vrednost razpoloženja (sentimenta).

Na koncu shranimo spletno stran v datoteko s pripono 'html'.

Datoteko lahko prenesemo na svoj računalnik in odpremo v brskalniku, npr. če v levem panelu odpremo mapo, izberemo 'Prenos' in v Raziskovalcu kliknemo html datoteko (privzeto jo najdemo v mapi 'Prenosi' / 'Downloads').



In [7]:
from IPython.core.display import display, HTML

# Map labels to colors
def get_color(label):
    if label == "POSITIVE":
        return "green"
    elif label == "NEGATIVE":
        return "red"
    return "gray"  # Neutral

# Create an HTML string with proper formatting
html = """<!DOCTYPE html>
<html>
<head>
    <title>Sentiment Analysis</title>
    <style>
        body { font-family: Arial, sans-serif; line-height: 1.8; margin: 20px; }
        .positive { color: green; }
        .neutral { color: gray; }
        .negative { color: red; }
    </style>
</head>
<body>
<h1>Sentiment Analysis Results</h1>
<p>Below are the sentences from the document, colored based on their sentiment:</p>
"""

for result in results:
    label = result["sentiment"]["label"].lower()
    html += f"<p class='{label}'>{result['sentence']}</p>\n"

html += "</body></html>"

# Save the HTML to a file
output_file = "sentiment_analysis_colored.html"
with open(output_file, "w", encoding="utf-8") as file:
    file.write(html)

print(f"HTML file saved as {output_file}. Open it in a browser to view the results.")

HTML file saved as sentiment_analysis_colored.html. Open it in a browser to view the results.


C:\Users\teodo\AppData\Local\Temp\ipykernel_23204\189112583.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
